A model on $n$ vertices with order $d$.
$K$ vertices forming a clique, meaning that any size-d tuple within the clique is connected by a hyperedge.
All other size-$d$ tuples form a hyperedge with probability $q = 1/2$ .

In [876]:
import math
import time as tm
from itertools import permutations

import numpy as np

In [877]:
# set the degree of each vertex on graph to 1
# def add_allll(i, j, k, g):
#     g[np.array(list(set(permutations([i, j, k]))))] = 1
#     return g

# increment the degree of each vertex on graph g by 1
def add_all(i, j, k, g):
    g[i, j, k] = 1
    g[i, k, j] = 1
    g[j, i, k] = 1
    g[j, k, i] = 1
    g[k, i, j] = 1
    g[k, j, i] = 1
    return g


# set the degree of each vertex on graph g to 0
# def remove_all(i, j, k, g):
#     g[np.array(list(set(permutations([i, j, k]))))] = 0
#     return g

# decrement the degree of each vertex on graph g to 0
def remove_all(i, j, k, g):
    g[i, j, k] = 0
    g[i, k, j] = 0
    g[j, i, k] = 0
    g[j, k, i] = 0
    g[k, i, j] = 0
    g[k, j, i] = 0
    return g

In [878]:
### create a graph with planted clique

# g: graph (represented as a tensor)
# num_vertices: number of vertices on the graph (dimension, or rank of the tensor) 
# pr: probability of creating edges
# clq_size: clique size

def generate_graph(num_vertices, pr, clq_size):
    def create_edges():
        for i in range(num_vertices):
            for j in range(i + 1, num_vertices):
                for k in range(j + 1, num_vertices):
                    if np.random.uniform(0, 1, 1) < pr:
                        vec[[i, j, k]] += 1
                        add_all(i, j, k, g)

    def plant_clique():
        for i in range(clq_size):
            for j in range(i + 1, clq_size):
                for k in range(j + 1, clq_size):
                    vec[[clq_vertex[i], clq_vertex[j], clq_vertex[k]]] += 1
                    add_all(clq_vertex[i], clq_vertex[j], clq_vertex[k], g)

    g = np.zeros((num_vertices, num_vertices, num_vertices))
    vec = np.zeros(num_vertices, )
    create_edges()
    clq_vertex = np.random.choice(range(num_vertices), clq_size, replace=False)
    plant_clique()

    return g, vec, clq_vertex

In [879]:
# calculate the total number of edges on a graph with num_vertices vertices
def count_edge(g, num_vertices):
    num_edges = 0
    for i in range(num_vertices):
        for j in range(i + 1, num_vertices):
            for k in range(j + 1, num_vertices):
                if g[i, j, k] == 1:
                    num_edges += 1
    return num_edges

In [880]:
# check if the graph is a clique
def is_clique(g, vec, num_vertices):
    active_count = np.count_nonzero(vec)  # number of vertices that are associated with at least 1 edge
    edge_sum = count_edge(g, num_vertices)  # number of edges
    return edge_sum == math.comb(active_count, 3)

In [881]:
# remove edge
def remove_edges(g, num_vertices, vec, curr_idx):
    vec[curr_idx] = 0
    for j in range(0, num_vertices):
        for k in range(0, num_vertices):
            if g[curr_idx, j, k] == 1:
                vec[[j, k]] -= 1
                remove_all(curr_idx, j, k, g)
    return g, vec

### Generate a graph
N: number of vertices
P: probability of an edge being included
K: clique size

G: graph with planted clique
V: vector storing the number of edges associated with each vertex in the graph
clique_vertices: the set of clique vertices

In [882]:
N = 100  # number of vertices
P = 0.5  # probability of an edge being included
K = 10  # clique size

start_generate = tm.time()
res = generate_graph(N, P, K)
G, V, clique_vertices = res[0], res[1], res[-1]
G_0 = G.copy()

print(f"edge-occurrence vector: {V}\n")
print(f"set of clique vertices after generation phase: {clique_vertices}\n")

time_generate = np.round(tm.time() - start_generate, 3)
# print("Time taken to generate graph: ", "seconds")

edge-occurrence vector: [2384. 2407. 2405. 2514. 2512. 2493. 2465. 2409. 2419. 2373. 2494. 2444.
 2445. 2439. 2350. 2457. 2490. 2429. 2476. 2412. 2428. 2385. 2409. 2455.
 2453. 2407. 2392. 2375. 2437. 2454. 2456. 2437. 2402. 2366. 2412. 2441.
 2409. 2398. 2467. 2466. 2436. 2401. 2415. 2389. 2353. 2383. 2466. 2473.
 2443. 2399. 2453. 2390. 2435. 2407. 2432. 2424. 2382. 2387. 2465. 2444.
 2488. 2419. 2433. 2419. 2386. 2455. 2380. 2418. 2437. 2406. 2482. 2462.
 2441. 2444. 2397. 2476. 2483. 2421. 2431. 2521. 2389. 2447. 2442. 2444.
 2348. 2411. 2433. 2470. 2368. 2382. 2468. 2373. 2465. 2457. 2427. 2465.
 2444. 2467. 2424. 2410.]

set of clique vertices after generation phase: [79 76 64  2 67  3 60  5 15 19]



### Removal Phase

In [883]:
start_removal = tm.time()
itr = 0
removed = []  # keep track of the vertices removed from the original graph to form a clique

while not is_clique(G, V, N):
    itr += 1
    curr = -1
    idx_sorted = np.argsort(V)

    for idx in range(N):
        if V[idx_sorted[idx]] != 0:
            curr = idx_sorted[idx]
            removed.append(curr)
            break

    A = remove_edges(G, N, V, curr)
    G = A[0]

print(f"is a clique at iteration #{itr}!!!")
time_remove = np.round(tm.time() - start_removal, 3)

is a clique at iteration #91!!!


In [884]:
print(f"number of iterations in the removal phase: {itr}")

number of iterations in the removal phase: 91


In [885]:
included = np.setdiff1d(np.arange(N), removed)
# included

### Inclusion Phase

In [886]:
def inclusion_phase(target, in_set, g):
    def connected():
        for j in range(len(in_set)):
            for k in range(j + 1, len(in_set)):
                if g[target_idx, in_set[j], in_set[k]] != 1:
                    # print(f" vertex {idx} is not connected to all clique vertices")
                    return False
        return True

    for target_idx in target:
        if connected():
            print(f"add {target_idx} to clique!")
            in_set = np.append(in_set, target_idx)
            print(in_set)

    return in_set

In [887]:
start_include = tm.time()
res = inclusion_phase(removed, included, G_0)
time_include = np.round(tm.time() - start_include, 3)

add 64 to clique!
[ 2  3  5 15 19 60 67 76 79 64]


In [888]:
print(f"Set of clique vertices after generation phase: {np.sort(clique_vertices)}\n")
print(f"Set of clique vertices after removal phase: {np.sort(included)}\n")
print(f"Set of clique vertices after inclusion phase: {np.sort(res)}")

Set of clique vertices after generation phase: [ 2  3  5 15 19 60 64 67 76 79]

Set of clique vertices after removal phase: [ 2  3  5 15 19 60 67 76 79]

Set of clique vertices after inclusion phase: [ 2  3  5 15 19 60 64 67 76 79]
